In [1]:
import ultralytics
from ultralytics import YOLO, solutions
#import supervision as sv
#import torch
import pandas as pd
import plotly.express as px
from datetime import datetime
import time

In [ ]:
#uncomment line below if using wandb.ai for experiment tracking
#ultralytics.settings.update({"wandb" : True })

In [ ]:
class_name = "thalassiosira"
project = "IFCB Chain Counter"
name = f"IFCBChainCounts_{class_name}_n"

TRAIN

In [ ]:
#model = YOLO("YOLO11n-obb.yaml").load("yolo11n.pt") 
#model = YOLO("YOLO11s.yaml").load("yolo11s.pt") 
model = YOLO("yolo11x.pt") 


#results = model.train(data="datasets/thalasiossira_rotated/data.yaml", epochs=200, project=project_name)
results = model.train(data=f"datasets/{class_name}/data.yaml", epochs=200, project=project, name=name)

PREDICT

In [3]:
#load model
model = YOLO(f"IFCB Chain Counter/{name}/weights/best.pt")
#model_normal_m = YOLO("IFCBChainCounts_Thalasiossira_normal_m/train/weights/best.pt")

In [ ]:
df = pd.read_csv(f'datasets/{class_name}/images/test/test_image_counts.csv')


iou_range = 0.0
#f = open(project_name + '/results/overall_results.csv', 'w')

overall_results = pd.DataFrame(columns=['model_name', 'iou_range', 'total_difference'])


for i in range(0,40):

    print("Iteration:" , i)
    print("IOU_Value:" , iou_range)

    count_score = []

    for index, row in df.iterrows():
        #print(row['file_name'], " Cell Count: ", row['cell_count'])

        results = model([f'datasets/{class_name}/images/test/'+ row['file_name']], iou=iou_range, verbose=False)
        
        #get occurences in image from results and save output image
        for result in results:
            boxes = result.boxes  # Boxes object for bounding box outputs
            probs = result.probs  # Probs object for classification outputs
        
            count_score.append(len(result.boxes))
            #TODO: Only need to save files for the best run
            #result.save(filename="IFCB Chain Counter/" + name + f"/results/result_{row['file_name']}")
        
    
    
    test_results = df
    test_results['output_count'] = count_score
    test_results['difference'] = test_results['cell_count'] - test_results['output_count']

    total_difference = test_results['difference'].abs().sum()

    print("Total Difference is: " , total_difference)

    file_to_save = "IFCB Chain Counter/" + name + f"/results/{name}_output_counts_iou_{iou_range}.csv"
    test_results.to_csv(file_to_save, index=False)

    new_result_dict = pd.DataFrame.from_dict({"model_name" : [name], "iou_range": [iou_range],"total_difference": [total_difference]})
    
    overall_results = pd.concat([overall_results, new_result_dict], ignore_index=True)
    

    #f.close()

    iou_range += 0.025



overall_results.to_csv("IFCB Chain Counter/" + name + '/results/overall_results.csv', index=False, mode='w', lineterminator='\n')
#f.close()

In [ ]:
df1 = pd.read_csv(f'IFCB Chain Counter/IFCBChainCounts_{class_name}_x/results/overall_results.csv')
df2 = pd.read_csv(f'IFCB Chain Counter/IFCBChainCounts_{class_name}_l/results/overall_results.csv')
df3 = pd.read_csv(f'IFCB Chain Counter/IFCBChainCounts_{class_name}_m/results/overall_results.csv')
df4 = pd.read_csv(f'IFCB Chain Counter/IFCBChainCounts_{class_name}_s/results/overall_results.csv')
df5 = pd.read_csv(f'IFCB Chain Counter/IFCBChainCounts_{class_name}_n/results/overall_results.csv')

df = pd.concat([df1,df2,df3,df4,df5]) 
df['model_name'] = df['model_name'].str.replace('IFCBChainCounts_', '')

print(df)
fig = px.line(df, x='iou_range', log_y=True, y='total_difference', color='model_name', title='Chaetoceros - IOU Accuracy')

fig.update_layout(
    title=dict(
        text="Thalassiosira IOU Accuracy"
    ),
    xaxis=dict(
        title=dict(
            text="IOU Thresholds"
        )
    ),
    yaxis=dict(
        title=dict(
            text="Difference in Counts"
        )
    ),
    legend=dict(
        title=dict(
            text=""
        ),
        #visible=False
    ),
    width=700,
    height=700,
    font=dict(
        #family="Courier New, monospace",
        size=20,
        #color="RebeccaPurple"
    )
)
fig.show()

              model_name  iou_range  total_difference
0   thalassiosira_1000_x      0.000             510.0
1   thalassiosira_1000_x      0.025             344.0
2   thalassiosira_1000_x      0.050             233.0
3   thalassiosira_1000_x      0.075             174.0
4   thalassiosira_1000_x      0.100             139.0
..                   ...        ...               ...
35  thalassiosira_1000_n      0.875             162.0
36  thalassiosira_1000_n      0.900             178.0
37  thalassiosira_1000_n      0.925             221.0
38  thalassiosira_1000_n      0.950             334.0
39  thalassiosira_1000_n      0.975             968.0

[200 rows x 3 columns]


OPTIMAL IOU


| Genus | Model | Optimal IOU | Total Instances | Difference | 
|:--------:|:--------:|:--------:|:--------:|:--------:|
|  Chaetoceros   |  Nano  |  0.325   | 3809 | 186 |
|  Chaetoceros   |  Small  |  0.4   | 3809 | 224 |
|  Chaetoceros   |  Medium   |  0.375   | 3809 | 210 |
|  Chaetoceros   |  Large  |  0.375   |3809 | 224 |
|  Chaetoceros   |  Xtra Large   |  0.375   | 3809 | 241|
|  Pseudo-Nitzschia | Nano | 0.125 to 0.475 | 1177 | 10 |
|  Pseudo-Nitzschia | Small | 0.625 to 0.725 | 1177 | 10 |
|  Pseudo-Nitzschia | Medium | 0.125 to 0.325 | 1177 | 12 |
|  Pseudo-Nitzschia | Large | 0.35 - 0.4 | 1177 | 8 |
|  Pseudo-Nitzschia | Xtra Large | 0.425 | 1177 | 9 |
|  Skeletonema   |  Nano  |  0.525 to 0.65   | 1826 | 7 |
|  Skeletonema   |  Small  |  0.025   | 1826 | 5 |
|  Skeletonema   |  Medium   |  0.025   | 1826 | 8 |
|  Skeletonema   |  Large  |  0.025   | 1826 | 7 |
|  Skeletonema   |  Xtra Large   |  0.025   | 1826 | 9 |
|  Thalassiosira | Nano | 0.175 | 3295 | 91 |
|  Thalassiosira | Small | 0.25 and 0.45 | 3295 | 61 |
|  Thalassiosira | Medium | 0.3 to 0.325 | 3295 | 63 |
|  Thalassiosira | Large | 0.225 4 | 3295 | 72 |
|  Thalassiosira | Xtra Large | 0.325 to 0.35 | 3295 | 79 |




Skeletonema Nano - Best IOU: 0.525 to 0.65 Diff: 11
Skeletonema Small - Best IOU 0.025 Diff: 9
Skeletonema Medium - Best IOU 0.025 Diff: 12
Skeletonema Large - Best IOU 0.025 - 0.05 Diff: 11
Skeletonema Xtra Large - Best IOU 0.025 - 0.05 Diff: 13

Chaetoceros Nano - Best IOU: 0.325 Diff: 212
Chaetoceros Small - Best IOU 0.4 and 0.45 Diff: 238
Chaetoceros Medium - Best IOU 0.375 Diff: 226
Chaetoceros Large - Best IOU 0.375 Diff: 236
Chaetoceros Xtra Large - Best IOU 0.375 Diff: 255

In [ ]:
optimal_iou_for_class = 0.325

df = pd.read_csv(f'datasets/{class_name}/images/test/test_image_counts.csv')

count_score = []
times = []
#start_time = datetime.now()
#total_infer_time = 0
for index, row in df.iterrows():
    #print(row['file_name'], " Cell Count: ", row['cell_count'])
    start_time = time.time()
    results = model([f"datasets/{class_name}/images/test/"+ row["file_name"]], iou=optimal_iou_for_class, verbose=False)
    #infer_time_end = datetime.now()
    end_time = time.time()
    times.append(end_time - start_time)
    #get occurences in image from results and save output image
    for result in results:
        boxes = result.boxes  # Boxes object for bounding box outputs
        probs = result.probs  # Probs object for classification outputs
    
        count_score.append(len(result.boxes))
        
        #annotated_img = result.plot(line_width=1, font_size=10)
        result.save(filename="IFCB Chain Counter/" + name + f"/results/result_{row['file_name']}", line_width=1)
        #annotated_img.show()
        #annotated_img.save(filename="IFCB Chain Counter/" + name + f"/results/annotated_img_{row['file_name']}")
    
test_results = df
test_results['output_count'] = count_score
test_results['difference'] = test_results['cell_count'] - test_results['output_count']
test_results['time'] = times


total_difference = test_results['difference'].abs().sum()
total_time = test_results['time'].sum()

print("Total Difference is: " , total_difference)
print("Total Time to Infer is: " , total_time)
file_to_save = "IFCB Chain Counter/" + name + f"/results/{name}_output_counts_final.csv"
test_results.to_csv(file_to_save, index=False)



    






Total Difference is:  186
Total Time to Infer is:  26.337639331817627


| Genus | Model | Run 1 | Run 2 | Run 3 | Avg | 
|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|
|Chaetoceros |Nano| 8.798026084899902| 8.30193281173706| 8.786971807479858| 8.628976901|
|Chaetoceros |Small| 8.649919271469116| 9.316676616668701| 8.257943868637085| 8.741513252|
|Chaetoceros |Medium| 9.346074104309082| 11.148280143737793| 11.080913066864014| 10.5250891|
|Chaetoceros |Large| 13.531863689422607| 15.499467611312866| 14.070124626159668| 14.36715198|
|Chaetoceros |Xtra large| 12.930631399154663| 14.235334634780884| 13.70563817024231| 13.62386807|
|Pseudo-Nitzschia |Nano| 7.298567771911621| 7.747407913208008| 7.6361403465271| 7.560705344|
|Pseudo-Nitzschia |Small| 8.061251640319824| 7.467475414276123| 8.227312326431274| 7.918679794|
|Pseudo-Nitzschia |Medium| 9.42411184310913| 9.25805115699768| 9.486964464187622|  9.389709155|
|Pseudo-Nitzschia |Large| 13.5796537399292| 13.227951765060425| 12.159743547439575| 12.98911635|
|Pseudo-Nitzschia |Xtra large| 13.237974166870117|  11.832287788391113| 14.361827850341797| 13.14402994|
|Skeletonema |Nano| 7.862987279891968| 7.515233516693115| 8.05016827583313| 7.809463024 |
|Skeletonema |Small| 7.864756345748901| 7.239608287811279|  7.467054128646851| 7.523806254 |
|Skeletonema |Medium|  8.652581930160522| 9.1401526927948| 9.087902545928955| 8.96021239 |
|Skeletonema |Large| 12.601239681243896| 12.296869277954102| 13.649321556091309| 12.84914351 |
|Skeletonema |Xtra Large| 14.671318054199219| 12.892750263214111|  11.769779682159424| 13.11128267 |
|Thalassiosira |Nano| 9.330669403076172| 8.533891439437866| 8.335193872451782| 8.733251572 |
|Thalassiosira |Small| 8.654560565948486| 9.869890689849854|9.012516975402832| 9.17898941 |
|Thalassiosira |Medium| 10.763086080551147| 10.753660440444946| 9.914618015289307| 10.47712151 |
|Thalassiosira |Large| 15.487650394439697|  14.836585283279419| 14.296731948852539| 14.87365588 |
|Thalassiosira |Xlarge|   15.210422039031982| 14.428626775741577| 14.856074810028076| 14.83170787 |



